There are several versions of models in this file, as we started with simple models and kept adding features in each version. The final version of the model is at the bottom of this file, along with some comparison and analysis.

In [ ]:
!pip install simpy

In [ ]:
import numpy as np
import simpy
from scipy.stats import uniform, t, norm, expon, poisson
import time

One Lane Model

In [ ]:
def arrivals(env, servers, S, G):
    j=0
    while True:
        # Arrival rate
        dt=expon.rvs(scale=1)

        if env.now + dt > 70:        # time to stop accepting new arrivals
          break
        else:
          arrival = env.timeout(dt)
          yield arrival
          S.append(env.now)
          env.process(service(env, servers, S, G, j))
          j+=1
        
def service(env, servers, S, G, j):
    n=len(servers)
    for i in range(0, n):
        if i == 0:                            # enter the first segment if possible
          rqt = servers[i].request()
          yield rqt
        else:
          rqt = new_rqt
        #servicetime = expon.rvs(scale=1)
        if i == n - 1:                        # if we are at the end of highway
          servicetime = uniform.rvs()*0.2+0.4
        elif servers[i+1].count:              # if we have traffic in front of us
          servicetime = uniform.rvs()*0.4+0.8
        else:                                 # if we have space in front
          servicetime = uniform.rvs()*0.2+0.4
        service = env.timeout(servicetime)
        yield service
        if i != n - 1:                        # if there is space in front, move to next server; otherwise, remain waiting in the current server
          new_rqt = servers[i+1].request()
          yield new_rqt
        servers[i].release(rqt)
        #print('car ', j, ' goes through ', i, ' at time ', env.now)

    G.append(env.now)
    
def simulate(n):
    S = []
    G = []
    env = simpy.Environment()
    servers=[simpy.Resource(env, capacity=1) for _ in range(n)]
    env.process(arrivals(env, servers, S, G))
    env.run()
    elapsed_time = np.array(G) - np.array(S)
    print('Average time for a vehicle to pass the highway is: ', np.mean(elapsed_time))
    print('Average speed :'+ str(round(1000/np.mean(elapsed_time),3))+'m/s')
    return #S, G

# simulate for 1km (100 servers)
simulate(100)

Average time for a vehicle to pass the highway is:  58.33846095909075
Average speed :17.141m/s


# 2 lanes model

In [ ]:
def arrivals(env, servers, S, G,road):
    j=0
    while True:
        # Arrival rate
        dt=expon.rvs(scale=0.5)
        if env.now + dt > 70:        # time to stop accepting new arrivals
          break
        else:
          arrival = env.timeout(dt)
          yield arrival
          S.append(env.now)
          env.process(service(env, servers, S, G, j,road))
          j+=1
        
def service(env, servers, S, G, j,road):
    n=len(servers[0])
    for i in range(0, n):
        if i == 0:                            # enter the first segment if possible
          if uniform.rvs() * 2 < 1:           # choose the first lane
            lane = 0
            rqt = servers[lane][i].request()
          else:                               # choose the second lane
            lane = 1
            rqt = servers[lane][i].request()
          #print('car ', j, ' enters lane ', lane)
          yield rqt
        else:
          lane = newlane
          rqt = new_rqt
        #servicetime = expon.rvs(scale=1)
        if i == n - 1:                              # if we are at the end of highway
          servicetime = uniform.rvs()*0.2+0.4
        elif servers[lane][i+1].count:              # if we have traffic in front of us
          servicetime = uniform.rvs()*0.4+0.8
        else:                                       # if we have space in front
          servicetime = uniform.rvs()*0.2+0.4
        service = env.timeout(servicetime)
        yield service
        if i != n - 1:
          if not servers[lane][i+1].count:              # checking car in front in our own line
            newlane = lane                              # if no car we stay in line
            new_rqt = servers[lane][i+1].request()
            yield new_rqt
          else:
            if servers[1-lane][i+1].count + servers[1-lane][i].count == 0:  # if no car on our lef/right and no car
              newlane = 1-lane                                  #  in front in the other line we switch
              new_rqt = servers[1-lane][i+1].request()
              yield new_rqt
            elif servers[1-lane][i].count == 0:                           # if no car on the other lane in i, we wait to see which lane is faster in i+1 to see if we switch or not
              new_rqt1, new_rqt2 = servers[lane][i+1].request(), servers[1-lane][i+1].request() #we make a request to both servers in segment i+1 and goes to the first one available
              result = yield new_rqt1 | new_rqt2 
              if new_rqt1 in result:
                new_rqt2.cancel()
                newlane = lane
                new_rqt = new_rqt1
              else:
                new_rqt1.cancel()
                newlane = 1 - lane
                new_rqt = new_rqt2
            else: 
              newlane = lane                              # if cars everywhere, we stay in line 
              new_rqt = servers[lane][i+1].request()
              yield new_rqt

              
        if servers[0][i].count==1:
            road[i][1]=' o '
        elif servers[1][i].count==1:
            road[i][2]=' o '
        elif servers[0][i].count==0:
            road[i][1]='  '
        elif servers[1][i].count==0:
            road[i][2]='  '
        #print(road)
        #time.sleep(1)
        servers[lane][i].release(rqt)
    G.append(env.now)

def simulate(n):
    S = []
    G = []
    road=np.array(['  ' for i in range(3*n)]).reshape(n,3)
    for i in range(1,n+1):
        road[i-1][0]=i
        
    env = simpy.Environment()
    servers1=[simpy.Resource(env, capacity=1) for _ in range(n)]
    servers2=[simpy.Resource(env, capacity=1) for _ in range(n)]
    servers = [servers1, servers2]
    env.process(arrivals(env, servers, S, G,road))
    env.run()
    elapsed_time = np.array(G) - np.array(S)
    print('Average time for a vehicle to pass the highway is: ', np.mean(elapsed_time))
    print('Average speed :'+ str(round(1000/np.mean(elapsed_time),3))+'m/s')
    return #S, G

"""def simulate(n):
    S = []
    G = []
    env = simpy.Environment()
    servers1=[simpy.Resource(env, capacity=1) for _ in range(n)]
    servers2=[simpy.Resource(env, capacity=1) for _ in range(n)]
    servers = [servers1, servers2]
    env.process(arrivals(env, servers, S, G))
    env.run()
    elapsed_time = np.array(G) - np.array(S)
    print('Average time for a vehicle to pass the highway is: ', np.mean(elapsed_time))
    print('Average speed :'+ str(round(1000/np.mean(elapsed_time),3))+'m/s')
    return #S, G"""

# simulate for 1km (100 servers)
np.random.seed(1)
simulate(100)

Average time for a vehicle to pass the highway is:  74.71157077610536
Average speed :13.385m/s


# N lanes

In [ ]:
def make_rqt_to_lines(new_rqts, result, corresponding_lanes):
    '''
    if the list new_rqts=[rqt1, rqt2],
    the list corresponding_lanes should be equal to [lane1, lane2] 
    with lane1 corresponding to rqt1 and lane2 to rqt2
    '''
    for i in range(len(new_rqts)):
        if new_rqts[i] in result:
            #cancel the other requests
            for j in range(len(new_rqts)):
                if j!=i:
                    new_rqts[j].cancel()
            newlane = corresponding_lanes[i]
            new_rqt = new_rqts[i]
    return new_rqt, newlane

def arrivals(env, servers, S, G):
    j=0
    while True:
        # Arrival rate
        dt=expon.rvs(scale=0.5)
        if env.now + dt > 70:        # time to stop accepting new arrivals
            break
        else:
            arrival = env.timeout(dt)
            yield arrival
            S.append(env.now)
            env.process(service(env, servers, S, G, j))
            j+=1
        
def service(env, servers, S, G, j):
    n=len(servers[0])
    for i in range(0, n):
        if i == 0:
            u=uniform.rvs()*len(servers)            # rondom initial position
            lane=int(u)
            rqt = servers[lane][i].request()
            yield rqt
        else:
            lane = newlane
            rqt = new_rqt
        #servicetime = expon.rvs(scale=1)
        if i == n - 1:                              # if we are at the end of highway
            servicetime = uniform.rvs()*0.2+0.4
        elif servers[lane][i+1].count:              # if we have traffic in front of us
            servicetime = uniform.rvs()*0.4+0.8     # service time longer -> we slow down
        else:                                       # if we have space in front
            servicetime = uniform.rvs()*0.2+0.4     # service time shorter -> we speed up
        service = env.timeout(servicetime)
        yield service
        if i != n - 1:
            if not servers[lane][i+1].count:              # checking car in front in our own line
                newlane = lane                              # if no car we stay in line
                new_rqt = servers[lane][i+1].request()    # we move forward in our lane 
                yield new_rqt
            else:
                if lane == 0: #specific case, if we are on lane 0, we can only move to line 0 or 1
                    if servers[lane+1][i+1].count + servers[lane+1][i].count == 0:
                        newlane = lane+1
                        new_rqt = servers[lane+1][i+1].request()
                        yield new_rqt
                    elif servers[lane+1][i].count == 0:                          # if no car on the other lane in i, we wait to see which lane is faster in i+1 to see if we switch or not
                        new_rqt1, new_rqt2 = servers[lane][i+1].request(), servers[lane+1][i+1].request() 
                        result = yield new_rqt1 | new_rqt2 
                        new_rqt, newlane = make_rqt_to_lines([new_rqt1, new_rqt2], result, [lane, lane+1])
                    else :                              # if cars everywhere, we stay in line 
                        newlane = lane
                        new_rqt = servers[lane][i+1].request()
                        yield new_rqt

                        
                elif lane == len(servers)-1: #specific case, if we are the last lane, we can only move to line n-1 (stay on the last) or n-2
                    if servers[lane-1][i+1].count + servers[lane-1][i].count == 0:
                        newlane = lane-1
                        new_rqt = servers[lane-1][i+1].request()
                        yield new_rqt
                    elif servers[lane-1][i].count == 0:   # if no car on the other lane in i, we wait to see which lane is faster in i+1 to see if we switch or not
                        new_rqt1, new_rqt2 = servers[lane][i+1].request(), servers[lane-1][i+1].request() 
                        result = yield new_rqt1 | new_rqt2 
                        new_rqt, newlane = make_rqt_to_lines([new_rqt1, new_rqt2], result, [lane, lane-1])
                    else:                     # if cars everywhere, we stay in line 
                        newlane = lane
                        new_rqt = servers[lane][i+1].request()
                        yield new_rqt

                else:
                    if (servers[lane-1][i+1].count + servers[lane-1][i].count == 0) and\
                       (servers[lane+1][i+1].count + servers[lane+1][i].count == 0): #if both lines i+1 and i-1 are available, we randomly choose one
                        if uniform.rvs()<0.5:
                            newlane = lane-1
                        else:
                            newlane = lane+1
                        new_rqt = servers[newlane][i+1].request()
                        yield new_rqt
                    elif servers[lane-1][i+1].count + servers[lane-1][i].count == 0:   #if only lane bellow is free
                        newlane = lane-1
                        new_rqt = servers[newlane][i+1].request()
                        yield new_rqt
                    elif servers[lane+1][i+1].count + servers[lane+1][i].count == 0: #if only lane above is free
                        newlane = lane+1
                        new_rqt = servers[newlane][i+1].request()
                        yield new_rqt
                    elif servers[lane-1][i].count == 0 and servers[lane+1][i].count == 0 :   # if no car on lane-1 and lane+1 in i, we wait to see which lane is faster betwenn lane-1, lane+1 and lane in i+1 to see if we switch or not
                      #we make a request to the three servers in segment i, i-1 and i+1 and goes to the first one available                           
                        new_rqts = [servers[lane-1][i+1].request(), servers[lane+1][i+1].request(), servers[lane][i+1].request()]    # !!!! need to check on server[i] first if we can move above or bellow....
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane-1, lane+1, lane])
                    elif servers[lane-1][i].count == 0:   # if no car on lane-1 in i, we wait to see which lane is faster betwenn lane-1 and lane in i+1 to see if we switch or not
                      #we make a request to the three servers in segment i, i-1 and i+1 and goes to the first one available                           
                        new_rqts = [servers[lane-1][i+1].request(), servers[lane][i+1].request()]
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane-1, lane])
                    elif servers[lane+1][i].count == 0:   # if no car on lane+1 in i, we wait to see which lane is faster betwenn lane-1 and lane in i+1 to see if we switch or not
                      #we make a request to the three servers in segment i, i-1 and i+1 and goes to the first one available                           
                        new_rqts = [servers[lane+1][i+1].request(), servers[lane][i+1].request()]
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane+1, lane])
                    else:
                        newlane = lane                              # if we cannot switch because cars in lane-1 and lane +1 in i
                        new_rqt = servers[lane][i+1].request()      # we move forward in our lane 
                        yield new_rqt


        servers[lane][i].release(rqt)
        #print('car ', j, ' goes through ', i, 'on lane', lane ,' at time ', env.now)

    G.append(env.now)
    
def simulate(n, lanes_nb):
    S = []
    G = []
    env = simpy.Environment()
    servers = [[simpy.Resource(env, capacity=1) for _ in range(n)] for _ in range(lanes_nb)]
    env.process(arrivals(env, servers, S, G))
    env.run()
    elapsed_time = np.array(G) - np.array(S)
    print('Average time for a vehicle to pass the highway is: {:.2f} s'.format(np.mean(elapsed_time)))
    print('Average speed : {:.2f} m/s'.format(1000/np.mean(elapsed_time)))
    return #S, G

# simulate for 1km (100 servers)
np.random.seed(1)
simulate(100, 3)

Average time for a vehicle to pass the highway is: 54.28 s
Average speed : 18.42 m/s


# Adding Priority to request

In [ ]:
def make_rqt_to_lines(new_rqts, result, corresponding_lanes):
    '''
    if the list new_rqts=[rqt1, rqt2],
    the list corresponding_lanes should be equal to [lane1, lane2] 
    with lane1 corresponding to rqt1 and lane2 to rqt2
    '''
    for i in range(len(new_rqts)):
        if new_rqts[i] in result:
            #cancel the other requests
            for j in range(len(new_rqts)):
                if j!=i:
                    new_rqts[j].cancel()
            newlane = corresponding_lanes[i]
            new_rqt = new_rqts[i]
    return new_rqt, newlane

def arrivals(env, servers, arrival_rate, S, G):
    j=0
    while True:
        # Arrival rate
        dt=expon.rvs(scale=arrival_rate)
        if env.now + dt > 70:        # time to stop accepting new arrivals
            break
        else:
            arrival = env.timeout(dt)
            yield arrival
            S.append(env.now)
            env.process(service(env, servers, S, G, j))
            j+=1
        
def service(env, servers, S, G, j):
    n=len(servers[0])
    for i in range(0, n):
        if i == 0:
            u=uniform.rvs()*len(servers)            # rondom initial position
            lane=int(u)
            rqt = servers[lane][i].request()
            yield rqt
        else:
            lane = newlane
            rqt = new_rqt
        #servicetime = expon.rvs(scale=1)
        if i == n - 1:                              # if we are at the end of highway
            servicetime = uniform.rvs()*0.2+0.4
        elif servers[lane][i+1].count:              # if we have traffic in front of us
            servicetime = uniform.rvs()*0.4+0.8     # service time longer -> we slow down
        else:                                       # if we have space in front
            servicetime = uniform.rvs()*0.2+0.4     # service time shorter -> we speed up
        service = env.timeout(servicetime)
        yield service
        if i != n - 1:
            if not servers[lane][i+1].count:              # checking car in front in our own line
                newlane = lane                            # if no car we stay in line
                new_rqt = servers[lane][i+1].request(priority = -1)    # we move forward in our lane 
                yield new_rqt
            else:
                if lane == 0: #specific case, if we are on lane 0, we can only move to line 0 or 1
                    if servers[lane+1][i+1].count + servers[lane+1][i].count == 0:
                        newlane = lane+1
                        new_rqt = servers[lane+1][i+1].request(priority = 0)
                        yield new_rqt
                    elif servers[lane+1][i].count == 0:                          # if no car on the other lane in i, we wait to see which lane is faster in i+1 to see if we switch or not
                        new_rqt1, new_rqt2 = servers[lane][i+1].request(priority = -1), servers[lane+1][i+1].request(priority = 0) 
                        result = yield new_rqt1 | new_rqt2 
                        new_rqt, newlane = make_rqt_to_lines([new_rqt1, new_rqt2], result, [lane, lane+1])
                    else :                              # if cars everywhere, we stay in line 
                        newlane = lane
                        new_rqt = servers[lane][i+1].request(priority = -1)
                        yield new_rqt

                        
                elif lane == len(servers)-1: #specific case, if we are the last lane, we can only move to line n-1 (stay on the last) or n-2
                    if servers[lane-1][i+1].count + servers[lane-1][i].count == 0:
                        newlane = lane-1
                        new_rqt = servers[lane-1][i+1].request(priority = 0)
                        yield new_rqt
                    elif servers[lane-1][i].count == 0: 
                        new_rqt1, new_rqt2 = servers[lane][i+1].request(priority = -1), servers[lane-1][i+1].request(priority = 0) 
                        result = yield new_rqt1 | new_rqt2 
                        new_rqt, newlane = make_rqt_to_lines([new_rqt1, new_rqt2], result, [lane, lane-1])
                    else:
                        newlane = lane
                        new_rqt = servers[lane][i+1].request(priority = -1)
                        yield new_rqt
                    

                else:
                    if (servers[lane-1][i+1].count + servers[lane-1][i].count == 0) and\
                       (servers[lane+1][i+1].count + servers[lane+1][i].count == 0): #if both lines i+1 and i-1 are available, we randomly choose one
                        if uniform.rvs()<0.5:
                            newlane = lane-1
                        else:
                            newlane = lane+1
                        new_rqt = servers[newlane][i+1].request(priority = 0)
                        yield new_rqt
                    elif servers[lane-1][i+1].count + servers[lane-1][i].count == 0:   #if only lane bellow is free
                        newlane = lane-1
                        new_rqt = servers[newlane][i+1].request(priority = 0)
                        yield new_rqt
                    elif servers[lane+1][i+1].count + servers[lane+1][i].count == 0: #if only lane above is free
                        newlane = lane+1
                        new_rqt = servers[newlane][i+1].request(priority = 0)
                        yield new_rqt
                    elif servers[lane-1][i].count == 0 and servers[lane+1][i].count == 0 :   # if no car on lane-1 and lane+1 in i, we wait to see which lane is faster betwenn lane-1, lane+1 and lane in i+1 to see if we switch or not
                      #we make a request to the three servers in segment i, i-1 and i+1 and goes to the first one available                           
                        new_rqts = [servers[lane-1][i+1].request(priority = 0), servers[lane+1][i+1].request(priority = 0), servers[lane][i+1].request(priority = -1)]    # !!!! need to check on server[i] first if we can move above or bellow....
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane-1, lane+1, lane])
                    elif servers[lane-1][i].count == 0:   # if no car on lane-1 in i, we wait to see which lane is faster betwenn lane-1 and lane in i+1 to see if we switch or not
                      #we make a request to the three servers in segment i, i-1 and i+1 and goes to the first one available                           
                        new_rqts = [servers[lane-1][i+1].request(priority = 0), servers[lane][i+1].request(priority = -1)]
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane-1, lane])
                    elif servers[lane+1][i].count == 0:   # if no car on lane+1 in i, we wait to see which lane is faster betwenn lane-1 and lane in i+1 to see if we switch or not
                      #we make a request to the three servers in segment i, i-1 and i+1 and goes to the first one available                           
                        new_rqts = [servers[lane+1][i+1].request(priority = 0), servers[lane][i+1].request(priority = -1)]
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane+1, lane])
                    else:
                        newlane = lane                              # if we cannot switch because cars in lane-1 and lane +1 in i
                        new_rqt = servers[lane][i+1].request(priority = -1)      # we move forward in our lane 
                        yield new_rqt


        servers[lane][i].release(rqt)
        #print('car ', j, ' goes through ', i, 'on lane', lane ,' at time ', env.now)

    G.append(env.now)
    
def simulate(n, arrival_rate, lanes_nb):
    S = []
    G = []
    env = simpy.Environment()
    servers = [[simpy.PriorityResource(env, capacity=1) for _ in range(n)] for _ in range(lanes_nb)]
    env.process(arrivals(env, servers, arrival_rate, S, G))
    env.run()
    elapsed_time = np.array(G) - np.array(S)
    print('Average time for a vehicle to pass the highway is: {:.2f} s'.format(np.mean(elapsed_time)))
    print('Average speed : {:.2f} m/s'.format(1000/np.mean(elapsed_time)))
    return #S, G

# simulate for 1km (100 servers)
np.random.seed(1)
simulate(100, 0.33, 3)

Average time for a vehicle to pass the highway is: 68.27 s
Average speed : 14.65 m/s


## Optimization

Adding cost/revenue to the model and generating profit.\
Assumptions: \
Cost : The first lane is around \\$1.75M/km. Any additional lanes is about \\$1M/km \
Profit : \\$0.08125/km  (\\$0.13/mile) \
Function of lambda vs. time: $\frac{1}{5}(2sin(\frac{1}{6}\pi(\frac{t}{10}-4))+3) $ \
Function of lamba vs. nb_lanes: lambda is proportional to $\sqrt{lanes}$ (bigger roads, bigger traffic).\
Toll Booth employee salary : \$20/h * nb of lanes \
The construction fee will be repaid in 20 years, with loan interest rate 4% per year \
Maintaince fee will be \\$40K per year per km for each lane \

$\lambda(t) = \frac{\sin \left( \pi \frac{\frac{t}{10}-4}{6}\right) +3}{5}\times \sqrt{lanes} $

$\Lambda(t) = \left(\frac{3t}{5}- \frac{12 \cos \left( \pi \frac{\frac{t}{10}-4}{6}\right)}{\pi} - \frac{6}{\pi} \right)\times \sqrt{lanes}$

$\frac{6}{\pi}$ is to make $\Lambda(0)=0$

In [ ]:
x_range=np.linspace(0, 240, 100)
lambda_t = lmbda(x_range, 3)

import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12,7))
ax.grid(True)
ax.plot(x_range, lambda_t, linewidth=2, label='$\lambda(t)$')
ax.set_xlabel('time')
ax.set_ylabel('arrival rate $\lambda(t)$')
ax.fill_between(x_range, 0, 2, where=(lambda_t>1.2), color='gold', alpha=0.2, label='Rush hours')
ax.legend(loc=0)

NameError: ignored

In [ ]:
#return elapsed time

def simulate_with_elapsed_time(lanes_nb, is_print=False):
    #print(lanes_nb)
    n=100
    S = {}
    G = {}
    revenue=[]

    r = 0.15      # loan interest rate
    year = 20     # number of years to pay the debt
    loan = 1750000 + 1000000 * (lanes_nb - 1)
    year_loan = r * loan / (1+r) / (1-(1/(1+r))**year) 
    maintainance = 40000 * lanes_nb
    daily_cost = (year_loan + maintainance) / 365 + 24 * 20 * lanes_nb #20$ is the hourly salary of each employee (1 per lane)

    #NHPP
    max_period=240            # simulating 240 seconds
    nhpp_arrivals = thinning(lmbda, max_period, lanes_nb)
    nhpp_arrivals = np.insert(nhpp_arrivals,0,0)
    interarrivals = np.diff(nhpp_arrivals)

    service_times = []

    env = simpy.Environment()
    servers = [[simpy.PriorityResource(env, capacity=1) for _ in range(n)] for _ in range(lanes_nb)]
    env.process(arrivals(env, servers, S, G,revenue, interarrivals, is_print))
    env.process(construction(env, servers, is_print))
    env.run()
    elapsed_time = np.array([G[x]-S[x] for x in G])
    time_of_arrival = np.array([S[x] for x in G])

    if is_print:
      print('Average time for a vehicle to pass the highway is: {:.2f} s'.format(np.mean(elapsed_time)))
      print('Average speed : {:.2f} m/s'.format(1000/np.mean(elapsed_time)))
      print('Income : ${:.2f}'.format(sum(revenue)*360))
      print('Daily cost : ${:.2f}'.format(daily_cost))

    return sum(revenue)*360 - daily_cost, len(interarrivals), elapsed_time, time_of_arrival

np.random.seed(3)
revenue, number_of_arrivales, elapsed_time, time_of_arrival = simulate_with_elapsed_time(4)

In [ ]:
pos=np.argsort(time_of_arrival)
time_of_arrival=time_of_arrival[pos]
elapsed_time=elapsed_time[pos]
fig, ax = plt.subplots(figsize=(12,7))
ax.grid(True)
ax.plot(time_of_arrival, elapsed_time, linewidth=2, label='time spent on the highway')
ax.set_xlabel('time of arrival')
ax.set_ylabel('time spent')
ax.fill_between(x_range, 48, 58, where=(lambda_t>1.2), color='gold', alpha=0.2, label='Rush hours')
ax.legend(loc=0)

In [ ]:
def make_rqt_to_lines(new_rqts, result, corresponding_lanes):
    '''
    if the list new_rqts=[rqt1, rqt2],
    the list corresponding_lanes should be equal to [lane1, lane2] 
    with lane1 corresponding to rqt1 and lane2 to rqt2
    '''
    for i in range(len(new_rqts)):
        if new_rqts[i] in result:
            #cancel the other requests
            for j in range(len(new_rqts)):
                if j!=i:
                    new_rqts[j].cancel()
            newlane = corresponding_lanes[i]
            new_rqt = new_rqts[i]
    return new_rqt, newlane
  
def lmbda(t, lanes):
    return (2*np.sin(np.pi*(t/10-4)/6) + 3)/5*(np.sqrt(lanes))
  
def thinning(lmbda, T, lanes):
    lmbda_max = lanes
    N = poisson.rvs(lmbda_max*T)
    ts = uniform.rvs(size=N) * T
    ts = np.sort(ts)
    S = []
    for t in ts:
        p = lmbda(t, lanes) / lmbda_max
        if uniform.rvs() < p:
            S.append(t)
    return np.array(S)

def arrivals(env, servers, S, G, revenue):
    j=0
    lanes = len(servers)
    max_period=240            # simulating 240 seconds
    arrivals = thinning(lmbda, max_period, lanes)
    arrivals = np.insert(arrivals,0,0)
    interarrivals = np.diff(arrivals)
  
    for interarrival in interarrivals:

      dt = env.timeout(interarrival)
      yield dt
      S.append(env.now)
      env.process(service(env, servers, S, G, j,revenue))
      j+=1
    
    #print('Number of cars in a day :', j*360)  

def service(env, servers, S, G, j,revenue):
    n=len(servers[0])
    for i in range(0, n):
        if i == 0:
            u=uniform.rvs()*len(servers)            # rondom initial position
            lane=int(u)
            rqt = servers[lane][i].request()
            yield rqt
        else:
            lane = newlane
            rqt = new_rqt
        #servicetime = expon.rvs(scale=1)
        if i == n - 1:                              # if we are at the end of highway
            servicetime = uniform.rvs()*0.2+0.4
        elif i==np.floor(n/2)-1:
            servicetime = uniform.rvs()*3+2         # 3-5 seconds to go through toll booth
            revenue.append(0.08125) 
        elif servers[lane][i+1].count:              # if we have traffic in front of us
            servicetime = uniform.rvs()*0.4+0.8     # service time longer -> we slow down
        else:                                       # if we have space in front
            servicetime = uniform.rvs()*0.2+0.4     # service time shorter -> we speed up
        service = env.timeout(servicetime)

        yield service
        if i != n - 1:
            if not servers[lane][i+1].count:              # checking car in front in our own line
                newlane = lane                            # if no car we stay in line
                new_rqt = servers[lane][i+1].request(priority = -1)    # we move forward in our lane 
                yield new_rqt
            else:
                if lane == 0: #specific case, if we are on lane 0, we can only move to line 0 or 1
                    if servers[lane+1][i+1].count + servers[lane+1][i].count == 0:
                        newlane = lane+1
                        new_rqt = servers[lane+1][i+1].request(priority = 0)
                        yield new_rqt
                    elif servers[lane+1][i].count == 0:                          # if no car on the other lane in i, we wait to see which lane is faster in i+1 to see if we switch or not
                        new_rqt1, new_rqt2 = servers[lane][i+1].request(priority = -1), servers[lane+1][i+1].request(priority = 0) 
                        result = yield new_rqt1 | new_rqt2 
                        new_rqt, newlane = make_rqt_to_lines([new_rqt1, new_rqt2], result, [lane, lane+1])
                    else :                              # if cars everywhere, we stay in line 
                        newlane = lane
                        new_rqt = servers[lane][i+1].request(priority = -1)
                        yield new_rqt

                        
                elif lane == len(servers)-1: #specific case, if we are the last lane, we can only move to line n-1 (stay on the last) or n-2
                    if servers[lane-1][i+1].count + servers[lane-1][i].count == 0:
                        newlane = lane-1
                        new_rqt = servers[lane-1][i+1].request(priority = 0)
                        yield new_rqt
                    elif servers[lane-1][i].count == 0: 
                        new_rqt1, new_rqt2 = servers[lane][i+1].request(priority = -1), servers[lane-1][i+1].request(priority = 0) 
                        result = yield new_rqt1 | new_rqt2 
                        new_rqt, newlane = make_rqt_to_lines([new_rqt1, new_rqt2], result, [lane, lane-1])
                    else:
                        newlane = lane
                        new_rqt = servers[lane][i+1].request(priority = -1)
                        yield new_rqt
                    

                else:
                    if (servers[lane-1][i+1].count + servers[lane-1][i].count == 0) and\
                       (servers[lane+1][i+1].count + servers[lane+1][i].count == 0): #if both lines i+1 and i-1 are available, we randomly choose one
                        if uniform.rvs()<0.5:
                            newlane = lane-1
                        else:
                            newlane = lane+1
                        new_rqt = servers[newlane][i+1].request(priority = 0)
                        yield new_rqt
                    elif servers[lane-1][i+1].count + servers[lane-1][i].count == 0:   #if only lane bellow is free
                        newlane = lane-1
                        new_rqt = servers[newlane][i+1].request(priority = 0)
                        yield new_rqt
                    elif servers[lane+1][i+1].count + servers[lane+1][i].count == 0: #if only lane above is free
                        newlane = lane+1
                        new_rqt = servers[newlane][i+1].request(priority = 0)
                        yield new_rqt
                    elif servers[lane-1][i].count == 0 and servers[lane+1][i].count == 0 :   # if no car on lane-1 and lane+1 in i, we wait to see which lane is faster betwenn lane-1, lane+1 and lane in i+1 to see if we switch or not
                      #we make a request to the three servers in segment i, i-1 and i+1 and goes to the first one available                           
                        new_rqts = [servers[lane-1][i+1].request(priority = 0), servers[lane+1][i+1].request(priority = 0), servers[lane][i+1].request(priority = -1)]    # !!!! need to check on server[i] first if we can move above or bellow....
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane-1, lane+1, lane])
                    elif servers[lane-1][i].count == 0:   # if no car on lane-1 in i, we wait to see which lane is faster betwenn lane-1 and lane in i+1 to see if we switch or not
                      #we make a request to the three servers in segment i, i-1 and i+1 and goes to the first one available                           
                        new_rqts = [servers[lane-1][i+1].request(priority = 0), servers[lane][i+1].request(priority = -1)]
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane-1, lane])
                    elif servers[lane+1][i].count == 0:   # if no car on lane+1 in i, we wait to see which lane is faster betwenn lane-1 and lane in i+1 to see if we switch or not
                      #we make a request to the three servers in segment i, i-1 and i+1 and goes to the first one available                           
                        new_rqts = [servers[lane+1][i+1].request(priority = 0), servers[lane][i+1].request(priority = -1)]
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane+1, lane])
                    else:
                        newlane = lane                              # if we cannot switch because cars in lane-1 and lane +1 in i
                        new_rqt = servers[lane][i+1].request(priority = -1)      # we move forward in our lane 
                        yield new_rqt


        servers[lane][i].release(rqt)
        #print('car ', j, ' goes through ', i, 'on lane', lane ,' at time ', env.now)

    G.append(env.now)
    
def simulate(lanes_nb):
    #print(lanes_nb)
    n=100
    S = []
    G = []
    revenue=[]

    r = 0.15      # loan interest rate
    year = 20     # number of years to pay the debt
    loan = 1750000 + 1000000 * (lanes_nb - 1)
    year_loan = r * loan / (1+r) / (1-(1/(1+r))**year)       # Use cash flow
    maintainance = 40000 * lanes_nb
    daily_cost = (year_loan + maintainance) / 365 + 24 * 20 * lanes_nb

    env = simpy.Environment()
    servers = [[simpy.PriorityResource(env, capacity=1) for _ in range(n)] for _ in range(lanes_nb)]
    env.process(arrivals(env, servers, S, G,revenue))
    env.run()
    elapsed_time = np.array(G) - np.array(S)

    #print('Average time for a vehicle to pass the highway is: {:.2f} s'.format(np.mean(elapsed_time)))
    #print('Average speed : {:.2f} m/s'.format(1000/np.mean(elapsed_time)))
    #print('Income : ${:.2f}'.format(sum(revenue)*360))
    #print('Daily cost : ${:.2f}'.format(daily_cost))

    return sum(revenue)*360 - daily_cost #,S, G

simulate(4)

In [ ]:
import hyperopt
def eval_mean(lanes_nb):
    if type(lanes_nb)==tuple:
        lanes_nb=int(lanes_nb[0])
    np.random.seed(42)
    X = [simulate(lanes_nb) for i in range(0, 10)]
    return -np.mean(X)

In [ ]:
best = hyperopt.fmin(fn=eval_mean,
    space=[hyperopt.hp.quniform('n1', 2, 6, 1)],
    algo=hyperopt.tpe.suggest,
    max_evals=5)
print(best)

## **Construction**

There will be random construction on the freeway. The construction will take up one block in one lane, and no two constructions will take place at the same time. The construction takes 60 seconds to finish, and the lane will be available

In [ ]:
def make_rqt_to_lines(new_rqts, result, corresponding_lanes):
    '''
    if the list new_rqts=[rqt1, rqt2],
    the list corresponding_lanes should be equal to [lane1, lane2] 
    with lane1 corresponding to rqt1 and lane2 to rqt2
    '''
    for i in range(len(new_rqts)):
        if new_rqts[i] in result:
            #cancel the other requests
            for j in range(len(new_rqts)):
                if j!=i:
                    new_rqts[j].cancel()
            newlane = corresponding_lanes[i]
            new_rqt = new_rqts[i]
            break
    return new_rqt, newlane
  
def lmbda(t, lanes):
    return (2*np.sin(np.pi*(t/10-4)/6) + 3)/5*(np.sqrt(lanes))

def thinning(lmbda, T, lanes):
    lmbda_max = lanes
    N = poisson.rvs(lmbda_max*T)
    ts = uniform.rvs(size=N) * T
    ts = np.sort(ts)
    S = []
    for t in ts:
        p = lmbda(t, lanes) / lmbda_max
        if uniform.rvs() < p:
            S.append(t)
    return np.array(S)

def construction(env, servers):
    lanes = len(servers)
    n = len(servers[0])
    max_period=240
    while True:
        dt = expon.rvs(scale=120)
        if env.now + dt > 2*max_period:
            break
        yield env.timeout(dt)
        accident_lane = int(uniform.rvs() * lanes)
        accident_location = int(uniform.rvs() * n)
        while accident_location in [0, 1, np.floor(n/2)-1, np.floor(n/2)]:
            accident_location = int(uniform.rvs() * n)
        server = servers[accident_lane][accident_location]

        rqt=server.request(priority=-10)
        yield rqt
        print('accident happens at {},{} at time {}'.format(accident_lane, accident_location, env.now))
        yield env.timeout(60)
        print('accident cleared at time {}'.format(env.now))
        server.release(rqt)    

def arrivals(env, servers, S, G, revenue):
    j=0
    lanes = len(servers)
    max_period=240            # simulating 240 seconds
    arrivals = thinning(lmbda, max_period, lanes)
    arrivals = np.insert(arrivals,0,0)
    interarrivals = np.diff(arrivals)
  
    for interarrival in interarrivals:

      dt = env.timeout(interarrival)
      yield dt
      S.append(env.now)
      env.process(service(env, servers, S, G, j,revenue))
      j+=1
    
    print('Number of cars in a day :', j*360)
           

def service(env, servers, S, G, j,revenue):
    n = len(servers[0])
    i = 0
    while i < n:
        if i == 0:
            u=uniform.rvs()*len(servers)            # random initial position
            lane=int(u)
            rqt = servers[lane][i].request(priority = -1)
            #print('car ', j, 'start on lane ', lane)
            yield rqt
        else:
            lane = newlane
            rqt = new_rqt

        #servicetime = expon.rvs(scale=1)
        if i == n - 1:                              # if we are at the end of highway
            servicetime = uniform.rvs()*0.2+0.4
        elif servers[lane][i+1].count:              # if we have traffic in front of us
            servicetime = uniform.rvs()*0.4+0.8     # service time longer -> we slow down
        else:                                       # if we have space in front
            servicetime = uniform.rvs()*0.2+0.4     # service time shorter -> we speed up
        service = env.timeout(servicetime)
        yield service


        if i==np.floor(n/2)-1:
           revenue.append(0.08125)                  # Toll booth tax (average us)
        i_next = i + 1

        if i != n - 1:
            if not servers[lane][i+1].count:              # checking car in front in our own line
                newlane = lane                            # if no car we stay in line
                new_rqt = servers[lane][i+1].request(priority = -1)    # we move forward in our lane 
                yield new_rqt
            else:
                if lane == 0: #specific case, if we are on lane 0, we can only move to line 0 or 1
                    if servers[lane+1][i+1].count + servers[lane+1][i].count == 0:
                        newlane = lane+1
                        new_rqt = servers[lane+1][i+1].request(priority = 1)
                        yield new_rqt
                    elif servers[lane+1][i].count == 0 or i == 0:                          # if no car on the other lane in i, we wait to see which lane is faster in i+1 to see if we switch or not
                        new_rqt1, new_rqt2 = servers[lane][i+1].request(priority = -1), servers[lane+1][i+1].request(priority = 1) 
                        result = yield new_rqt1 | new_rqt2 
                        new_rqt, newlane = make_rqt_to_lines([new_rqt1, new_rqt2], result, [lane, lane+1])
                    else :                              # if cars everywhere, we stay in line or wait to change to the position on the right
                        new_rqt1, new_rqt2 = servers[lane][i+1].request(priority = -1), servers[lane+1][i].request(priority = 0) 
                        result = yield new_rqt1 | new_rqt2
                        if new_rqt2 in result:
                            i_next = i
                        new_rqt, newlane = make_rqt_to_lines([new_rqt1, new_rqt2], result, [lane, lane+1])

                        
                elif lane == len(servers)-1: #specific case, if we are the last lane, we can only move to line n-1 (stay on the last) or n-2
                    if servers[lane-1][i+1].count + servers[lane-1][i].count == 0:
                        newlane = lane-1
                        new_rqt = servers[lane-1][i+1].request(priority = 1)
                        yield new_rqt
                    elif servers[lane-1][i].count == 0 or i == 0: 
                        new_rqt1, new_rqt2 = servers[lane][i+1].request(priority = -1), servers[lane-1][i+1].request(priority = 1) 
                        result = yield new_rqt1 | new_rqt2 
                        new_rqt, newlane = make_rqt_to_lines([new_rqt1, new_rqt2], result, [lane, lane-1])
                    else:
                        new_rqt1, new_rqt2 = servers[lane][i+1].request(priority = -1), servers[lane-1][i].request(priority = 0) 
                        result = yield new_rqt1 | new_rqt2
                        if new_rqt2 in result:
                            i_next = i
                        new_rqt, newlane = make_rqt_to_lines([new_rqt1, new_rqt2], result, [lane, lane-1])
                    

                else:
                    if (servers[lane-1][i+1].count + servers[lane-1][i].count == 0) and\
                       (servers[lane+1][i+1].count + servers[lane+1][i].count == 0): #if both lines i+1 and i-1 are available, we randomly choose one
                        if uniform.rvs()<0.5:
                            newlane = lane-1
                        else:
                            newlane = lane+1
                        new_rqt = servers[newlane][i+1].request(priority = 1)
                        yield new_rqt
                    elif servers[lane-1][i+1].count + servers[lane-1][i].count == 0:   #if only lane bellow is free
                        newlane = lane-1
                        new_rqt = servers[newlane][i+1].request(priority = 1)
                        yield new_rqt
                    elif servers[lane+1][i+1].count + servers[lane+1][i].count == 0: #if only lane above is free
                        newlane = lane+1
                        new_rqt = servers[newlane][i+1].request(priority = 1)
                        yield new_rqt
                    elif servers[lane-1][i].count == 0 and servers[lane+1][i].count == 0 :   # if no car on lane-1 and lane+1 in i, we wait to see which lane is faster betwenn lane-1, lane+1 and lane in i+1 to see if we switch or not
                      #we make a request to the three servers in segment i, i-1 and i+1 and goes to the first one available                           
                        new_rqts = [servers[lane-1][i+1].request(priority = 1), servers[lane+1][i+1].request(priority = 1), servers[lane][i+1].request(priority = -1)]    # !!!! need to check on server[i] first if we can move above or bellow....
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane-1, lane+1, lane])
                    elif servers[lane-1][i].count == 0:   # if no car on lane-1 in i, we wait to see which lane is faster betwenn lane-1 and lane in i+1 to see if we switch or not
                      #we make a request to the three servers in segment i, i-1 and i+1 and goes to the first one available                           
                        new_rqts = [servers[lane-1][i+1].request(priority = 1), servers[lane][i+1].request(priority = -1)]
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane-1, lane])
                    elif servers[lane+1][i].count == 0:   # if no car on lane+1 in i, we wait to see which lane is faster betwenn lane-1 and lane in i+1 to see if we switch or not
                      #we make a request to the three servers in segment i, i-1 and i+1 and goes to the first one available                           
                        new_rqts = [servers[lane+1][i+1].request(priority = 1), servers[lane][i+1].request(priority = -1)]
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane+1, lane])
                    elif i == 0:
                        new_rqts = [servers[lane-1][i+1].request(priority = 1), servers[lane+1][i+1].request(priority = 1), servers[lane][i+1].request(priority = -1)]
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane-1, lane+1, lane])
                    else:
                        new_rqts = [servers[lane-1][i].request(priority = 0), servers[lane+1][i].request(priority = 0), servers[lane][i+1].request(priority = -1)]
                        result = yield simpy.AnyOf(env, new_rqts)
                        if new_rqts[2] not in result:
                            i_next = i
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane-1, lane+1, lane])
  

        servers[lane][i].release(rqt)
        i = i_next
        #print('car ', j, ' goes through ', i, 'on lane', lane ,' at time ', env.now)

    G.append(env.now)
    #print(j, 'end')
    
def simulate(lanes_nb):
    #print(lanes_nb)
    n=100
    S = []
    G = []
    revenue=[]

    r = 0.15      # loan interest rate
    year = 20     # number of years to pay the debt
    loan = 1750000 + 1000000 * (lanes_nb - 1)
    year_loan = r * loan / (1+r) / (1-(1/(1+r))**year) 
    maintainance = 40000 * lanes_nb
    daily_cost = (year_loan + maintainance) / 365 + 24 * 20 * lanes_nb #20$ is the hourly salary of each employee (1 per lane)

    env = simpy.Environment()
    servers = [[simpy.PriorityResource(env, capacity=1) for _ in range(n)] for _ in range(lanes_nb)]
    env.process(arrivals(env, servers, S, G,revenue))
    env.process(construction(env, servers))
    env.run()

    elapsed_time = np.array(G) - np.array(S)


    print('Average time for a vehicle to pass the highway is: {:.2f} s'.format(np.mean(elapsed_time)))
    print('Average speed : {:.2f} m/s'.format(1000/np.mean(elapsed_time)))
    print('Income : ${:.2f}'.format(sum(revenue)*360))
    print('Daily cost : ${:.2f}'.format(daily_cost))

    return sum(revenue)*360 - daily_cost #,S, G

simulate(4)

accident happens at 0,19 at time 57.068602893587624
accident cleared at time 117.06860289358762
accident happens at 1,24 at time 155.5862351119812
accident cleared at time 215.5862351119812
Number of cars in a day : 97200
accident happens at 2,61 at time 237.68964533727367
accident cleared at time 297.68964533727365
accident happens at 0,9 at time 336.16598949022136
accident cleared at time 396.16598949022136
Average time for a vehicle to pass the highway is: 52.70 s
Average speed : 18.98 m/s
Income : $7897.50
Daily cost : $4166.26


3731.2415499052604

In [ ]:
def make_rqt_to_lines(new_rqts, result, corresponding_lanes):
    '''
    if the list new_rqts=[rqt1, rqt2],
    the list corresponding_lanes should be equal to [lane1, lane2] 
    with lane1 corresponding to rqt1 and lane2 to rqt2
    '''
    for i in range(len(new_rqts)):
        if new_rqts[i] in result:
            #cancel the other requests
            for j in range(len(new_rqts)):
                if j!=i:
                    new_rqts[j].cancel()
            newlane = corresponding_lanes[i]
            new_rqt = new_rqts[i]
            #break #I think we should add a break here but another issue appear when I add a break.

    return new_rqt, newlane
  
def lmbda(t, lanes):
    return (2*np.sin(np.pi*(t/10-4)/6) + 3)/5*(np.sqrt(lanes))

def Lambda(t, lanes):
  return ((3*t)/5 - 12*np.cos(np.pi *(t/10 - 4)/6)/np.pi - 6/np.pi )*np.sqrt(lanes)

def thinning(lmbda, T, lanes):
    lmbda_max = lanes
    N = poisson.rvs(lmbda_max*T)
    ts = uniform.rvs(size=N) * T
    ts = np.sort(ts)
    S = []
    for t in ts:
        p = lmbda(t, lanes) / lmbda_max
        if uniform.rvs() < p:
            S.append(t)
    return np.array(S)

def construction(env, servers,construction_locations,dts,construction_lanes):
    k=0
    max_period=240
    while True:
        dt = dts[k]
        #print('dt',dt,dt+60)
        construction_lane = construction_lanes[k]
        construction_location = construction_locations[k]
        if env.now + dt > 2*max_period:
            break
        
        yield env.timeout(dt)
        server = servers[construction_lane][construction_location]

        rqt=server.request(priority=-10)
        yield rqt
        print('accident happens at {},{} at time {}'.format(construction_lane, construction_location, env.now))
        yield env.timeout(60)
        print('accident cleared at time {}'.format(env.now))
        server.release(rqt)
        k+=1
        print('env:',env.now)

     

def arrivals(env, servers, S, G,revenue, interarrivals,road,construction_locations,contruction_time,construction_lanes):
    j=0
    lanes = len(servers)
    #common random numbers
    random_first_lane = (uniform.rvs(size=len(interarrivals))*lanes).astype(int)
    servicetime_short = uniform.rvs(size=(len(servers[0]), len(interarrivals)))*0.2+0.4
    servicetime_long = uniform.rvs(size=(len(servers[0]), len(interarrivals)))*0.4+0.8
    servicetime_TB = uniform.rvs(size=(1, len(interarrivals)))*3+2  
    
    lane_choice = uniform.rvs(size=(len(servers[0]), len(interarrivals))) #in case we have 2 empty lane and we have to choose
    for interarrival in interarrivals:
      dt = env.timeout(interarrival)
      yield dt
      S.append(env.now)
      env.process(service(env, servers, S, G, j, revenue, random_first_lane, servicetime_short, servicetime_long,servicetime_TB, lane_choice,road,construction_locations,contruction_time,construction_lanes))
      j+=1
    
 #   print('Number of cars in a day :', j*360)
           

def service(env, servers, S, G, j, revenue, random_first_lane, servicetime_short, servicetime_long,servicetime_TB, lane_choice,road,construction_locations,contruction_time,construction_lanes):
    n = len(servers[0])
    i = 0
    while i < n:
        if i == 0:        
            lane = random_first_lane[j] # random initial position
            rqt = servers[lane][i].request(priority = -1)
            #print('car ', j, 'start on lane ', lane)
            yield rqt
        else:
            lane = newlane
            rqt = new_rqt

        if i == n - 1:                              # if we are at the end of highway
            servicetime = servicetime_short[i][j]
        elif i==np.floor(n/2)-1:
            servicetime = servicetime_TB[0][j]      # 3-5 seconds to go through toll booth
            revenue.append(0.08125)                 # Toll booth tax (average us)
        elif servers[lane][i+1].count:              # if we have traffic in front of us
            servicetime = servicetime_long[i][j]     # service time longer -> we slow down
        else:                                       # if we have space in front
            servicetime = servicetime_short[i][j]     # service time shorter -> we speed up
        service = env.timeout(servicetime)
        yield service
              
        i_next = i + 1

        if i != n - 1:
            if not servers[lane][i+1].count:              # checking car in front in our own line
                newlane = lane                            # if no car we stay in line
                new_rqt = servers[lane][i+1].request(priority = -1)    # we move forward in our lane 
                yield new_rqt
            else:
                if lane == 0: #specific case, if we are on lane 0, we can only move to line 0 or 1
                    if servers[lane+1][i+1].count + servers[lane+1][i].count == 0:
                        newlane = lane+1
                        new_rqt = servers[lane+1][i+1].request(priority = 1)
                        yield new_rqt
                    elif servers[lane+1][i].count == 0 or i == 0:                          # if no car on the other lane in i, we wait to see which lane is faster in i+1 to see if we switch or not
                        new_rqt1, new_rqt2 = servers[lane][i+1].request(priority = -1), servers[lane+1][i+1].request(priority = 1) 
                        result = yield new_rqt1 | new_rqt2 
                        new_rqt, newlane = make_rqt_to_lines([new_rqt1, new_rqt2], result, [lane, lane+1])
                    else :                              # if cars everywhere, we stay in line or wait to change to the position on the right
                        new_rqt1, new_rqt2 = servers[lane][i+1].request(priority = -1), servers[lane+1][i].request(priority = 0) 
                        result = yield new_rqt1 | new_rqt2
                        if new_rqt2 in result:
                            i_next = i
                        new_rqt, newlane = make_rqt_to_lines([new_rqt1, new_rqt2], result, [lane, lane+1])

                        
                elif lane == len(servers)-1: #specific case, if we are the last lane, we can only move to line n-1 (stay on the last) or n-2
                    if servers[lane-1][i+1].count + servers[lane-1][i].count == 0:
                        newlane = lane-1
                        new_rqt = servers[lane-1][i+1].request(priority = 1)
                        yield new_rqt
                    elif servers[lane-1][i].count == 0 or i == 0: 
                        new_rqt1, new_rqt2 = servers[lane][i+1].request(priority = -1), servers[lane-1][i+1].request(priority = 1) 
                        result = yield new_rqt1 | new_rqt2 
                        new_rqt, newlane = make_rqt_to_lines([new_rqt1, new_rqt2], result, [lane, lane-1])
                    else:
                        new_rqt1, new_rqt2 = servers[lane][i+1].request(priority = -1), servers[lane-1][i].request(priority = 0) 
                        result = yield new_rqt1 | new_rqt2
                        if new_rqt2 in result:
                            i_next = i
                        new_rqt, newlane = make_rqt_to_lines([new_rqt1, new_rqt2], result, [lane, lane-1])
                    

                else:
                    if (servers[lane-1][i+1].count + servers[lane-1][i].count == 0) and\
                       (servers[lane+1][i+1].count + servers[lane+1][i].count == 0): #if both lines i+1 and i-1 are available, we randomly choose one
                        if lane_choice[i][j]<0.5:
                            newlane = lane-1
                        else:
                            newlane = lane+1
                        new_rqt = servers[newlane][i+1].request(priority = 1)
                        yield new_rqt
                    elif servers[lane-1][i+1].count + servers[lane-1][i].count == 0:   #if only lane bellow is free
                        newlane = lane-1
                        new_rqt = servers[newlane][i+1].request(priority = 1)
                        yield new_rqt
                    elif servers[lane+1][i+1].count + servers[lane+1][i].count == 0: #if only lane above is free
                        newlane = lane+1
                        new_rqt = servers[newlane][i+1].request(priority = 1)
                        yield new_rqt
                    elif servers[lane-1][i].count == 0 and servers[lane+1][i].count == 0 :   # if no car on lane-1 and lane+1 in i, we wait to see which lane is faster betwenn lane-1, lane+1 and lane in i+1 to see if we switch or not
                      #we make a request to the three servers in segment i, i-1 and i+1 and goes to the first one available                           
                        new_rqts = [servers[lane-1][i+1].request(priority = 1), servers[lane+1][i+1].request(priority = 1), servers[lane][i+1].request(priority = -1)]    # !!!! need to check on server[i] first if we can move above or bellow....
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane-1, lane+1, lane])
                    elif servers[lane-1][i].count == 0:   # if no car on lane-1 in i, we wait to see which lane is faster betwenn lane-1 and lane in i+1 to see if we switch or not
                      #we make a request to the three servers in segment i, i-1 and i+1 and goes to the first one available                           
                        new_rqts = [servers[lane-1][i+1].request(priority = 1), servers[lane][i+1].request(priority = -1)]
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane-1, lane])
                    elif servers[lane+1][i].count == 0:   # if no car on lane+1 in i, we wait to see which lane is faster betwenn lane-1 and lane in i+1 to see if we switch or not
                      #we make a request to the three servers in segment i, i-1 and i+1 and goes to the first one available                           
                        new_rqts = [servers[lane+1][i+1].request(priority = 1), servers[lane][i+1].request(priority = -1)]
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane+1, lane])
                    elif i == 0:
                        new_rqts = [servers[lane-1][i+1].request(priority = 1), servers[lane+1][i+1].request(priority = 1), servers[lane][i+1].request(priority = -1)]
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane-1, lane+1, lane])
                    else:
                        new_rqts = [servers[lane-1][i].request(priority = 0), servers[lane+1][i].request(priority = 0), servers[lane][i+1].request(priority = -1)]
                        result = yield simpy.AnyOf(env, new_rqts)
                        if new_rqts[2] not in result:
                            i_next = i
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane-1, lane+1, lane])
  
        constru=False
        ind=None
        for kb9 in range(0,len(contruction_time)):
            if contruction_time[kb9][0]<=env.now<=contruction_time[kb9][1]:
                constru=True
                ind=kb9
                
        for k in range(len(servers)):
            if i!=np.floor(n/2)-1:
                if servers[k][i].count==1:
                    road[i][k+1]=' o '#if switched==False else ' x '
                elif servers[k][i].count==0:
                    road[i][k+1]='  '
        if constru:
            road[construction_locations[ind]][construction_lanes[ind]+1]=' X '
       
            

        print(road)
        time.sleep(0.1)
        
        servers[lane][i].release(rqt)
        i = i_next
        #print('car ', j, ' goes through ', i, 'on lane', lane ,' at time ', env.now)
   
    G.append(env.now)
    #print(j, 'end')
    
def simulate(lanes_nb):
    n=10
    S = []
    G = []
    revenue=[]
    road=np.array(['  ' for i in range((lanes_nb+1)*n)]).reshape(n,lanes_nb+1)
    for i in range(1,n+1):
        if i!=np.floor(n/2):
            road[i-1][0]=i if i >=10 else str(0)+str(i) 
        elif i==np.floor(n/2):
            road[i-1][0]='TB'  
            for j in range(1,lanes_nb+1):
              road[i-1][j]='--' 

    r = 0.15      # loan interest rate
    year = 20     # number of years to pay the debt
    loan = 1750000 + 1000000 * (lanes_nb - 1)
    year_loan = r * loan / (1+r) / (1-(1/(1+r))**year) 
    maintainance = 40000 * lanes_nb
    daily_cost = (year_loan + maintainance) / 365 + 24 * 20 * lanes_nb #20$ is the hourly salary of each employee (1 per lane)

    #NHPP
    max_period=240            # simulating 240 seconds
    nhpp_arrivals = thinning(lmbda, max_period, lanes_nb)
    nhpp_arrivals = np.insert(nhpp_arrivals,0,0)
    interarrivals = np.diff(nhpp_arrivals)

    service_times = []
    
    env = simpy.Environment()
    servers = [[simpy.PriorityResource(env, capacity=1) for _ in range(n)] for _ in range(lanes_nb)]
    
    lanes = len(servers)
    nn = len(servers[0])
    #In order to have common random numbers, we need to generate the random numbers before the while. Otherwise, depending on the configuraiton, more or less constructions
    #can happen before the first constructions, changing the random numbers generation
    #I chose to generate 20 constructions (there is usually 2 accident, it should be enough)
    dts = expon.rvs(scale=120, size=20)
    construction_lanes = (uniform.rvs(size=20)* lanes).astype(int)
    construction_locations = []
    for _ in range(20):
      construction_locations.append(int(uniform.rvs() * nn))
      while construction_locations[-1] in [0, 1, np.floor(nn/2)-1, np.floor(nn/2)]: #We want to avoid having a construction just at the beginning and at the tool booth place
          construction_locations[-1] = int(uniform.rvs() * nn)
    contruction_time=[(sum(dts[:i+1])+60*i,sum(dts[:i+1])+60*(i+1)) for i in range(0,len(dts))] 

        

    
    env.process(arrivals(env, servers, S, G,revenue, interarrivals,road,construction_locations,contruction_time,construction_lanes))
    env.process(construction(env, servers,construction_locations,dts,construction_lanes))
    env.run()

    elapsed_time = np.array(G) - np.array(S)


    print('Average time for a vehicle to pass the highway is: {:.2f} s'.format(np.mean(elapsed_time)))
    print('Average speed : {:.2f} m/s'.format(1000/np.mean(elapsed_time)))
    print('Income : ${:.2f}'.format(sum(revenue)*360))
    print('Daily cost : ${:.2f}'.format(daily_cost))

    return sum(revenue)*360 - daily_cost, len(interarrivals)
    #revenue, number of arrival
np.random.seed(10)
simulate(4)

[['01' ' o' '  ' '  ' '  ']
 ['02' '  ' '  ' '  ' '  ']
 ['03' '  ' '  ' '  ' '  ']
 ['04' '  ' '  ' '  ' '  ']
 ['TB' '--' '--' '--' '--']
 ['06' '  ' '  ' '  ' '  ']
 ['07' '  ' '  ' '  ' '  ']
 ['08' '  ' '  ' '  ' '  ']
 ['09' '  ' '  ' '  ' '  ']
 ['10' '  ' '  ' '  ' '  ']]
[['01' ' o' '  ' '  ' '  ']
 ['02' ' o' '  ' '  ' '  ']
 ['03' '  ' '  ' '  ' '  ']
 ['04' '  ' '  ' '  ' '  ']
 ['TB' '--' '--' '--' '--']
 ['06' '  ' '  ' '  ' '  ']
 ['07' '  ' '  ' '  ' '  ']
 ['08' '  ' '  ' '  ' '  ']
 ['09' '  ' '  ' '  ' '  ']
 ['10' '  ' '  ' '  ' '  ']]
[['01' ' o' '  ' '  ' '  ']
 ['02' ' o' '  ' '  ' '  ']
 ['03' ' o' '  ' '  ' '  ']
 ['04' '  ' '  ' '  ' '  ']
 ['TB' '--' '--' '--' '--']
 ['06' '  ' '  ' '  ' '  ']
 ['07' '  ' '  ' '  ' '  ']
 ['08' '  ' '  ' '  ' '  ']
 ['09' '  ' '  ' '  ' '  ']
 ['10' '  ' '  ' '  ' '  ']]
[['01' '  ' '  ' ' o' '  ']
 ['02' ' o' '  ' '  ' '  ']
 ['03' ' o' '  ' '  ' '  ']
 ['04' '  ' '  ' '  ' '  ']
 ['TB' '--' '--' '--' '--']
 ['06' '  ' '  ' 

KeyboardInterrupt: ignored

# ADDING COMMON RANDOM NUMBERS AND CONTROL VARIATE

In [ ]:
def make_rqt_to_lines(new_rqts, result, corresponding_lanes):
    '''
    if the list new_rqts=[rqt1, rqt2],
    the list corresponding_lanes should be equal to [lane1, lane2] 
    with lane1 corresponding to rqt1 and lane2 to rqt2
    '''
    for i in range(len(new_rqts)):
        if new_rqts[i] in result:
            #cancel the other requests
            for j in range(len(new_rqts)):
                if j!=i:
                    new_rqts[j].cancel()
            newlane = corresponding_lanes[i]
            new_rqt = new_rqts[i]
            break #I think we should add a break here but another issue appear when I add a break.

    return new_rqt, newlane
  
def lmbda(t, lanes):
    return (2*np.sin(np.pi*(t/10-4)/6) + 3)/5*(np.sqrt(lanes))

def Lambda(t, lanes):
  return ((3*t)/5 - 12*np.cos(np.pi *(t/10 - 4)/6)/np.pi - 6/np.pi )*np.sqrt(lanes)

def thinning(lmbda, T, lanes):
    lmbda_max = lanes
    N = poisson.rvs(lmbda_max*T)
    ts = uniform.rvs(size=N) * T
    ts = np.sort(ts)
    S = []
    for t in ts:
        p = lmbda(t, lanes) / lmbda_max
        if uniform.rvs() < p:
            S.append(t)
    return np.array(S)

def construction(env, servers, is_print):
    lanes = len(servers)
    n = len(servers[0])
    max_period=240
    #In order to have common random numbers, we need to generate the random numbers before the while. Otherwise, depending on the configuraiton, more or less constructions
    #can happen before the first constructions, changing the random numbers generation
    #I chose to generate 50 constructions (there is usually 2 accident, it should be enough)
    dts = expon.rvs(scale=120, size=50)
    construction_lanes = (uniform.rvs(size=50)* lanes).astype(int)
    construction_locations = []
    for _ in range(50):
      construction_locations.append(int(uniform.rvs() * n))
      while construction_locations[-1] in [0, 1, np.floor(n/2)-1, np.floor(n/2)]: #We want to avoid having a construction just at the beginning and at the tool booth place
          construction_locations[-1] = int(uniform.rvs() * n)
    k=0
    while True:
        dt = dts[k]
        
        construction_lane = construction_lanes[k]
        construction_location = construction_locations[k]

        if env.now + dt > 2*max_period:
            break
        yield env.timeout(dt)
        server = servers[construction_lane][construction_location]

        rqt=server.request(priority=-10)
        yield rqt
        if is_print:
          print('accident happens at {},{} at time {}'.format(construction_lane, construction_location, env.now))
        yield env.timeout(60)
        if is_print:
          print('accident cleared at time {}'.format(env.now))
        server.release(rqt)
        k+=1    

def arrivals(env, servers, S, G,revenue, interarrivals, is_print):
    j=0
    lanes = len(servers)
    #common random numbers
    random_first_lane = (uniform.rvs(size=len(interarrivals))*lanes).astype(int)
    servicetime_short = uniform.rvs(size=(len(servers[0]), len(interarrivals)))*0.2+0.4
    servicetime_long = uniform.rvs(size=(len(servers[0]), len(interarrivals)))*0.4+0.8
    lane_choice = uniform.rvs(size=(len(servers[0]), len(interarrivals))) #in case we have 2 empty lane and we have to choose
    for interarrival in interarrivals:
      dt = env.timeout(interarrival)
      yield dt
      S.append(env.now)
      env.process(service(env, servers, S, G, j, revenue, random_first_lane, servicetime_short, servicetime_long, lane_choice))
      j+=1
    if is_print:
      print('Number of cars in a day :', j*360)
           

def service(env, servers, S, G, j, revenue, random_first_lane, servicetime_short, servicetime_long, lane_choice):
    n = len(servers[0])
    i = 0
    while i < n:
        if i == 0:        
            lane = random_first_lane[j] # random initial position
            rqt = servers[lane][i].request(priority = -1)
            #print('car ', j, 'start on lane ', lane)
            yield rqt
        else:
            lane = newlane
            rqt = new_rqt

        if i == n - 1:                              # if we are at the end of highway
            servicetime = servicetime_short[i][j]
        elif servers[lane][i+1].count:              # if we have traffic in front of us
            servicetime = servicetime_long[i][j]     # service time longer -> we slow down
        else:                                       # if we have space in front
            servicetime = servicetime_short[i][j]     # service time shorter -> we speed up
        service = env.timeout(servicetime)
        yield service


        if i==np.floor(n/2)-1:
           revenue.append(0.08125)                  # Toll booth tax (average us)
        i_next = i + 1

        if i != n - 1:
            if not servers[lane][i+1].count:              # checking car in front in our own line
                newlane = lane                            # if no car we stay in line
                new_rqt = servers[lane][i+1].request(priority = -1)    # we move forward in our lane 
                yield new_rqt
            else:
                if lane == 0: #specific case, if we are on lane 0, we can only move to line 0 or 1
                    if servers[lane+1][i+1].count + servers[lane+1][i].count == 0:
                        newlane = lane+1
                        new_rqt = servers[lane+1][i+1].request(priority = 1)
                        yield new_rqt
                    elif servers[lane+1][i].count == 0 or i == 0:                          # if no car on the other lane in i, we wait to see which lane is faster in i+1 to see if we switch or not
                        new_rqt1, new_rqt2 = servers[lane][i+1].request(priority = -1), servers[lane+1][i+1].request(priority = 1) 
                        result = yield new_rqt1 | new_rqt2 
                        new_rqt, newlane = make_rqt_to_lines([new_rqt1, new_rqt2], result, [lane, lane+1])
                    else :                              # if cars everywhere, we stay in line or wait to change to the position on the right
                        new_rqt1, new_rqt2 = servers[lane][i+1].request(priority = -1), servers[lane+1][i].request(priority = 0) 
                        result = yield new_rqt1 | new_rqt2
                        if new_rqt2 in result:
                            i_next = i
                        new_rqt, newlane = make_rqt_to_lines([new_rqt1, new_rqt2], result, [lane, lane+1])

                        
                elif lane == len(servers)-1: #specific case, if we are the last lane, we can only move to line n-1 (stay on the last) or n-2
                    if servers[lane-1][i+1].count + servers[lane-1][i].count == 0:
                        newlane = lane-1
                        new_rqt = servers[lane-1][i+1].request(priority = 1)
                        yield new_rqt
                    elif servers[lane-1][i].count == 0 or i == 0: 
                        new_rqt1, new_rqt2 = servers[lane][i+1].request(priority = -1), servers[lane-1][i+1].request(priority = 1) 
                        result = yield new_rqt1 | new_rqt2 
                        new_rqt, newlane = make_rqt_to_lines([new_rqt1, new_rqt2], result, [lane, lane-1])
                    else:
                        new_rqt1, new_rqt2 = servers[lane][i+1].request(priority = -1), servers[lane-1][i].request(priority = 0) 
                        result = yield new_rqt1 | new_rqt2
                        if new_rqt2 in result:
                            i_next = i
                        new_rqt, newlane = make_rqt_to_lines([new_rqt1, new_rqt2], result, [lane, lane-1])
                    

                else:
                    if (servers[lane-1][i+1].count + servers[lane-1][i].count == 0) and\
                       (servers[lane+1][i+1].count + servers[lane+1][i].count == 0): #if both lines i+1 and i-1 are available, we randomly choose one
                        if lane_choice[i][j]<0.5:
                            newlane = lane-1
                        else:
                            newlane = lane+1
                        new_rqt = servers[newlane][i+1].request(priority = 1)
                        yield new_rqt
                    elif servers[lane-1][i+1].count + servers[lane-1][i].count == 0:   #if only lane bellow is free
                        newlane = lane-1
                        new_rqt = servers[newlane][i+1].request(priority = 1)
                        yield new_rqt
                    elif servers[lane+1][i+1].count + servers[lane+1][i].count == 0: #if only lane above is free
                        newlane = lane+1
                        new_rqt = servers[newlane][i+1].request(priority = 1)
                        yield new_rqt
                    elif servers[lane-1][i].count == 0 and servers[lane+1][i].count == 0 :   # if no car on lane-1 and lane+1 in i, we wait to see which lane is faster betwenn lane-1, lane+1 and lane in i+1 to see if we switch or not
                      #we make a request to the three servers in segment i, i-1 and i+1 and goes to the first one available                           
                        new_rqts = [servers[lane-1][i+1].request(priority = 1), servers[lane+1][i+1].request(priority = 1), servers[lane][i+1].request(priority = -1)]    # !!!! need to check on server[i] first if we can move above or bellow....
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane-1, lane+1, lane])
                    elif servers[lane-1][i].count == 0:   # if no car on lane-1 in i, we wait to see which lane is faster betwenn lane-1 and lane in i+1 to see if we switch or not
                      #we make a request to the three servers in segment i, i-1 and i+1 and goes to the first one available                           
                        new_rqts = [servers[lane-1][i+1].request(priority = 1), servers[lane][i+1].request(priority = -1)]
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane-1, lane])
                    elif servers[lane+1][i].count == 0:   # if no car on lane+1 in i, we wait to see which lane is faster betwenn lane-1 and lane in i+1 to see if we switch or not
                      #we make a request to the three servers in segment i, i-1 and i+1 and goes to the first one available                           
                        new_rqts = [servers[lane+1][i+1].request(priority = 1), servers[lane][i+1].request(priority = -1)]
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane+1, lane])
                    elif i == 0:
                        new_rqts = [servers[lane-1][i+1].request(priority = 1), servers[lane+1][i+1].request(priority = 1), servers[lane][i+1].request(priority = -1)]
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane-1, lane+1, lane])
                    else:
                        new_rqts = [servers[lane-1][i].request(priority = 0), servers[lane+1][i].request(priority = 0), servers[lane][i+1].request(priority = -1)]
                        result = yield simpy.AnyOf(env, new_rqts)
                        if new_rqts[2] not in result:
                            i_next = i
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane-1, lane+1, lane])
  

        servers[lane][i].release(rqt)
        i = i_next
        #print('car ', j, ' goes through ', i, 'on lane', lane ,' at time ', env.now)
    #print('car ', j, ' goes through the highway at time ', env.now)
    G.append(env.now)
    #print(j, 'end')
    
def simulate(lanes_nb, is_print=False):
    #print(lanes_nb)
    n=100
    S = []
    G = []
    revenue=[]

    r = 0.15      # loan interest rate
    year = 20     # number of years to pay the debt
    loan = 1750000 + 1000000 * (lanes_nb - 1)
    year_loan = r * loan / (1+r) / (1-(1/(1+r))**year) 
    maintainance = 40000 * lanes_nb
    daily_cost = (year_loan + maintainance) / 365 + 24 * 20 * lanes_nb #20$ is the hourly salary of each employee (1 per lane)

    #NHPP
    max_period=240            # simulating 240 seconds
    nhpp_arrivals = thinning(lmbda, max_period, lanes_nb)
    nhpp_arrivals = np.insert(nhpp_arrivals,0,0)
    interarrivals = np.diff(nhpp_arrivals)

    service_times = []

    env = simpy.Environment()
    servers = [[simpy.PriorityResource(env, capacity=1) for _ in range(n)] for _ in range(lanes_nb)]
    env.process(arrivals(env, servers, S, G,revenue, interarrivals, is_print))
    env.process(construction(env, servers, is_print))
    env.run()
    elapsed_time = np.array(G) - np.array(S)

    if is_print:
      print('Average time for a vehicle to pass the highway is: {:.2f} s'.format(np.mean(elapsed_time)))
      print('Average speed : {:.2f} m/s'.format(1000/np.mean(elapsed_time)))
      print('Income : ${:.2f}'.format(sum(revenue)*360))
      print('Daily cost : ${:.2f}'.format(daily_cost))

    return sum(revenue)*360 - daily_cost, len(interarrivals)
    #revenue, number of arrival

np.random.seed(10)
simulate(4, is_print=True)

accident happens at 1,99 at time 45.752130297516004
accident cleared at time 105.752130297516
Number of cars in a day : 108000
accident happens at 1,95 at time 317.4417288356874
accident cleared at time 377.4417288356874
Average time for a vehicle to pass the highway is: 51.55 s
Average speed : 19.40 m/s
Income : $8775.00
Daily cost : $4166.26


(4608.741549905268, 300)

In [ ]:
def eval_mean(n):
  np.random.seed(3)
  T=240
  res = np.array([simulate(n) for _ in range(10)])
  revenue = res[:, 0]
  interarrival = res[:, 1]
  c1 = -np.cov(revenue, interarrival)[0][1] / np.var(interarrival)
  revenue_control_variate = revenue + c1 * (interarrival - Lambda(T, n))
  return np.mean(revenue_control_variate), np.mean(revenue), np.var(revenue_control_variate), np.var(revenue)

import hyperopt
def eval_mean_for_hyperopt(lanes_nb):
    if type(lanes_nb)==tuple:
        lanes_nb=int(lanes_nb[0])
    res = np.array([simulate(lanes_nb) for _ in range(10)])
    np.random.seed(3)
    T=240
    revenue = res[:, 0]
    interarrival = res[:, 1]
    c1 = -np.cov(revenue, interarrival)[0][1] / np.var(interarrival)
    revenue_control_variate = revenue + c1 * (interarrival - Lambda(T, n))
    return -np.mean(revenue_control_variate)


In [ ]:
mean_variate, mean_classic, var_variate, var_classic = eval_mean(4)

In [ ]:
print('variance reduction = {:.2f} %'.format((1-var_variate/var_classic)*100))

variance reduction = 98.77 %


**Final Model**

In [ ]:
def make_rqt_to_lines(new_rqts, result, corresponding_lanes):
    '''
    if the list new_rqts=[rqt1, rqt2],
    the list corresponding_lanes should be equal to [lane1, lane2] 
    with lane1 corresponding to rqt1 and lane2 to rqt2
    '''
    for i in range(len(new_rqts)):
        if new_rqts[i] in result:
            #cancel the other requests
            for j in range(len(new_rqts)):
                if j!=i:
                    new_rqts[j].cancel()
            newlane = corresponding_lanes[i]
            new_rqt = new_rqts[i]
            break #I think we should add a break here but another issue appear when I add a break.

    return new_rqt, newlane
  
def lmbda(t, lanes):
    return (2*np.sin(np.pi*(t/10-4)/6) + 3)/5*(np.sqrt(lanes))

def Lambda(t, lanes):
  return ((3*t)/5 - 12*np.cos(np.pi *(t/10 - 4)/6)/np.pi - 6/np.pi )*np.sqrt(lanes)

def thinning(lmbda, T, lanes):
    lmbda_max = lanes
    N = poisson.rvs(lmbda_max*T)
    ts = uniform.rvs(size=N) * T
    ts = np.sort(ts)
    S = []
    for t in ts:
        p = lmbda(t, lanes) / lmbda_max
        if uniform.rvs() < p:
            S.append(t)
    return np.array(S)

def construction(env, servers, is_print):
    lanes = len(servers)
    n = len(servers[0])
    max_period=240
    #In order to have common random numbers, we need to generate the random numbers before the while. Otherwise, depending on the configuraiton, more or less constructions
    #can happen before the first constructions, changing the random numbers generation
    #I chose to generate 50 constructions (there is usually 2 accident, it should be enough)
    dts = expon.rvs(scale=120, size=50)
    construction_lanes = (uniform.rvs(size=50)* lanes).astype(int)
    construction_locations = []
    for _ in range(50):
      construction_locations.append(int(uniform.rvs() * n))
      while construction_locations[-1] in [0, 1, np.floor(n/2)-1, np.floor(n/2)]: #We want to avoid having a construction just at the beginning and at the tool booth place
          construction_locations[-1] = int(uniform.rvs() * n)
    k=0
    while True:
        dt = dts[k]
        
        construction_lane = construction_lanes[k]
        construction_location = construction_locations[k]

        if env.now + dt > 2*max_period:
            break
        yield env.timeout(dt)
        server = servers[construction_lane][construction_location]

        rqt=server.request(priority=-10)
        yield rqt
        if is_print:
          print('accident happens at {},{} at time {}'.format(construction_lane, construction_location, env.now))
        yield env.timeout(60)
        if is_print:
          print('accident cleared at time {}'.format(env.now))
        server.release(rqt)
        k+=1    

def arrivals(env, servers, S, G,revenue, interarrivals, is_print):
    j=0
    lanes = len(servers)
    #common random numbers
    random_first_lane = (uniform.rvs(size=len(interarrivals))*lanes).astype(int)
    servicetime_short = uniform.rvs(size=(len(servers[0]), len(interarrivals)))*0.2+0.4
    servicetime_long = uniform.rvs(size=(len(servers[0]), len(interarrivals)))*0.4+0.8
    lane_choice = uniform.rvs(size=(len(servers[0]), len(interarrivals))) #in case we have 2 empty lane and we have to choose
    for interarrival in interarrivals:
      dt = env.timeout(interarrival)
      yield dt
      S[j] = env.now
      env.process(service(env, servers, S, G, j, revenue, random_first_lane, servicetime_short, servicetime_long, lane_choice))
      j+=1
    if is_print:
      print('Number of cars in a day :', j*360)
           

def service(env, servers, S, G, j, revenue, random_first_lane, servicetime_short, servicetime_long, lane_choice):
    n = len(servers[0])
    i = 0
    while i < n:
        if i == 0:        
            lane = random_first_lane[j] # random initial position
            rqt = servers[lane][i].request(priority = -1)
            #print('car ', j, 'start on lane ', lane)
            yield rqt
        else:
            lane = newlane
            rqt = new_rqt

        if i == n - 1:                              # if we are at the end of highway
            servicetime = servicetime_short[i][j]
        elif servers[lane][i+1].count:              # if we have traffic in front of us
            servicetime = servicetime_long[i][j]     # service time longer -> we slow down
        else:                                       # if we have space in front
            servicetime = servicetime_short[i][j]     # service time shorter -> we speed up
        service = env.timeout(servicetime)
        yield service


        if i==np.floor(n/2)-1:
           revenue.append(0.08125)                  # Toll booth tax (average us)
        i_next = i + 1

        if i != n - 1:
            if not servers[lane][i+1].count:              # checking car in front in our own line
                newlane = lane                            # if no car we stay in line
                new_rqt = servers[lane][i+1].request(priority = -1)    # we move forward in our lane 
                yield new_rqt
            else:
                if lane == 0: #specific case, if we are on lane 0, we can only move to line 0 or 1
                    if servers[lane+1][i+1].count + servers[lane+1][i].count == 0:
                        newlane = lane+1
                        new_rqt = servers[lane+1][i+1].request(priority = 1)
                        yield new_rqt
                    elif servers[lane+1][i].count == 0 or i == 0:                          # if no car on the other lane in i, we wait to see which lane is faster in i+1 to see if we switch or not
                        new_rqt1, new_rqt2 = servers[lane][i+1].request(priority = -1), servers[lane+1][i+1].request(priority = 1) 
                        result = yield new_rqt1 | new_rqt2 
                        new_rqt, newlane = make_rqt_to_lines([new_rqt1, new_rqt2], result, [lane, lane+1])
                    else :                              # if cars everywhere, we stay in line or wait to change to the position on the right
                        new_rqt1, new_rqt2 = servers[lane][i+1].request(priority = -1), servers[lane+1][i].request(priority = 0) 
                        result = yield new_rqt1 | new_rqt2
                        if new_rqt2 in result:
                            i_next = i
                        new_rqt, newlane = make_rqt_to_lines([new_rqt1, new_rqt2], result, [lane, lane+1])

                        
                elif lane == len(servers)-1: #specific case, if we are the last lane, we can only move to line n-1 (stay on the last) or n-2
                    if servers[lane-1][i+1].count + servers[lane-1][i].count == 0:
                        newlane = lane-1
                        new_rqt = servers[lane-1][i+1].request(priority = 1)
                        yield new_rqt
                    elif servers[lane-1][i].count == 0 or i == 0: 
                        new_rqt1, new_rqt2 = servers[lane][i+1].request(priority = -1), servers[lane-1][i+1].request(priority = 1) 
                        result = yield new_rqt1 | new_rqt2 
                        new_rqt, newlane = make_rqt_to_lines([new_rqt1, new_rqt2], result, [lane, lane-1])
                    else:
                        new_rqt1, new_rqt2 = servers[lane][i+1].request(priority = -1), servers[lane-1][i].request(priority = 0) 
                        result = yield new_rqt1 | new_rqt2
                        if new_rqt2 in result:
                            i_next = i
                        new_rqt, newlane = make_rqt_to_lines([new_rqt1, new_rqt2], result, [lane, lane-1])
                    

                else:
                    if (servers[lane-1][i+1].count + servers[lane-1][i].count == 0) and\
                       (servers[lane+1][i+1].count + servers[lane+1][i].count == 0): #if both lines i+1 and i-1 are available, we randomly choose one
                        if lane_choice[i][j]<0.5:
                            newlane = lane-1
                        else:
                            newlane = lane+1
                        new_rqt = servers[newlane][i+1].request(priority = 1)
                        yield new_rqt
                    elif servers[lane-1][i+1].count + servers[lane-1][i].count == 0:   #if only lane bellow is free
                        newlane = lane-1
                        new_rqt = servers[newlane][i+1].request(priority = 1)
                        yield new_rqt
                    elif servers[lane+1][i+1].count + servers[lane+1][i].count == 0: #if only lane above is free
                        newlane = lane+1
                        new_rqt = servers[newlane][i+1].request(priority = 1)
                        yield new_rqt
                    elif servers[lane-1][i].count == 0 and servers[lane+1][i].count == 0 :   # if no car on lane-1 and lane+1 in i, we wait to see which lane is faster betwenn lane-1, lane+1 and lane in i+1 to see if we switch or not
                      #we make a request to the three servers in segment i, i-1 and i+1 and goes to the first one available                           
                        new_rqts = [servers[lane-1][i+1].request(priority = 1), servers[lane+1][i+1].request(priority = 1), servers[lane][i+1].request(priority = -1)]    # !!!! need to check on server[i] first if we can move above or bellow....
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane-1, lane+1, lane])
                    elif servers[lane-1][i].count == 0:   # if no car on lane-1 in i, we wait to see which lane is faster betwenn lane-1 and lane in i+1 to see if we switch or not
                      #we make a request to the three servers in segment i, i-1 and i+1 and goes to the first one available                           
                        new_rqts = [servers[lane-1][i+1].request(priority = 1), servers[lane][i+1].request(priority = -1)]
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane-1, lane])
                    elif servers[lane+1][i].count == 0:   # if no car on lane+1 in i, we wait to see which lane is faster betwenn lane-1 and lane in i+1 to see if we switch or not
                      #we make a request to the three servers in segment i, i-1 and i+1 and goes to the first one available                           
                        new_rqts = [servers[lane+1][i+1].request(priority = 1), servers[lane][i+1].request(priority = -1)]
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane+1, lane])
                    elif i == 0:
                        new_rqts = [servers[lane-1][i+1].request(priority = 1), servers[lane+1][i+1].request(priority = 1), servers[lane][i+1].request(priority = -1)]
                        result = yield simpy.AnyOf(env, new_rqts)
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane-1, lane+1, lane])
                    else:
                        new_rqts = [servers[lane-1][i].request(priority = 0), servers[lane+1][i].request(priority = 0), servers[lane][i+1].request(priority = -1)]
                        result = yield simpy.AnyOf(env, new_rqts)
                        if new_rqts[2] not in result:
                            i_next = i
                        new_rqt, newlane = make_rqt_to_lines(new_rqts, result, [lane-1, lane+1, lane])
  

        servers[lane][i].release(rqt)
        i = i_next
        #print('car ', j, ' goes through ', i, 'on lane', lane ,' at time ', env.now)
    #print('car ', j, ' goes through the highway at time ', env.now)
    G[j]=env.now
    #print(j, 'end')
    
def simulate(lanes_nb, is_print=False):
    #print(lanes_nb)
    n=100
    S = {}
    G = {}
    revenue=[]

    r = 0.15      # loan interest rate
    year = 20     # number of years to pay the debt
    loan = 1750000 + 1000000 * (lanes_nb - 1)
    year_loan = r * loan / (1+r) / (1-(1/(1+r))**year) 
    maintainance = 40000 * lanes_nb
    daily_cost = (year_loan + maintainance) / 365 + 24 * 20 * lanes_nb #20$ is the hourly salary of each employee (1 per lane)

    #NHPP
    max_period=240            # simulating 240 seconds
    nhpp_arrivals = thinning(lmbda, max_period, lanes_nb)
    nhpp_arrivals = np.insert(nhpp_arrivals,0,0)
    interarrivals = np.diff(nhpp_arrivals)

    service_times = []

    env = simpy.Environment()
    servers = [[simpy.PriorityResource(env, capacity=1) for _ in range(n)] for _ in range(lanes_nb)]
    env.process(arrivals(env, servers, S, G,revenue, interarrivals, is_print))
    env.process(construction(env, servers, is_print))
    env.run()
    elapsed_time = np.array([G[x]-S[x] for x in G])

    if is_print:
      print('Average time for a vehicle to pass the highway is: {:.2f} s'.format(np.mean(elapsed_time)))
      print('Average speed : {:.2f} m/s'.format(1000/np.mean(elapsed_time)))
      print('Income : ${:.2f}'.format(sum(revenue)*360))
      print('Daily cost : ${:.2f}'.format(daily_cost))

    return sum(revenue)*360 - daily_cost, len(interarrivals)
    #revenue, number of arrival

np.random.seed(10)
simulate(4, is_print=True)

accident happens at 1,99 at time 45.752130297516004
accident cleared at time 105.752130297516
Number of cars in a day : 108000
accident happens at 1,95 at time 317.4417288356874
accident cleared at time 377.4417288356874
Average time for a vehicle to pass the highway is: 51.55 s
Average speed : 19.40 m/s
Income : $8775.00
Daily cost : $4166.26


(4608.741549905268, 300)

In [ ]:
def eval_mean(n):
  np.random.seed(3)
  T=240
  res = np.array([simulate(n) for _ in range(10)])
  revenue = res[:, 0]
  interarrival = res[:, 1]
  c1 = -np.cov(revenue, interarrival)[0][1] / np.var(interarrival)
  revenue_control_variate = revenue + c1 * (interarrival - Lambda(T, n))
  return np.mean(revenue_control_variate), np.mean(revenue), np.var(revenue_control_variate), np.var(revenue)

import hyperopt
def eval_mean_for_hyperopt(lanes_nb):
    if type(lanes_nb)==tuple:
        lanes_nb=int(lanes_nb[0])
    res = np.array([simulate(lanes_nb) for _ in range(10)])
    np.random.seed(3)
    T=240
    revenue = res[:, 0]
    interarrival = res[:, 1]
    c1 = -np.cov(revenue, interarrival)[0][1] / np.var(interarrival)
    revenue_control_variate = revenue + c1 * (interarrival - Lambda(T, lanes_nb))
    return -np.mean(revenue_control_variate)


In [ ]:
mean_variate, mean_classic, var_variate, var_classic = eval_mean(4)
print('variance reduction = {:.2f} %'.format((1-var_variate/var_classic)*100))

variance reduction = 98.77 %


**Model Comparison**

In [ ]:
best = hyperopt.fmin(fn=eval_mean_for_hyperopt,
    space=[hyperopt.hp.quniform('n1', 2, 10, 1)],
    algo=hyperopt.tpe.suggest,
    max_evals=10)
print(best)

100%|██████████| 10/10 [01:57<00:00, 11.76s/it, best loss: -4288.616549905264]
{'n1': 4.0}


In [ ]:
#return speed, revenue

def simulate_with_speed(lanes_nb, is_print=False):
    #print(lanes_nb)
    n=100
    S = {}
    G = {}
    revenue=[]

    r = 0.15      # loan interest rate
    year = 20     # number of years to pay the debt
    loan = 1750000 + 1000000 * (lanes_nb - 1)
    year_loan = r * loan / (1+r) / (1-(1/(1+r))**year) 
    maintainance = 40000 * lanes_nb
    daily_cost = (year_loan + maintainance) / 365 + 24 * 20 * lanes_nb #20$ is the hourly salary of each employee (1 per lane)

    #NHPP
    max_period=240            # simulating 240 seconds
    nhpp_arrivals = thinning(lmbda, max_period, lanes_nb)
    nhpp_arrivals = np.insert(nhpp_arrivals,0,0)
    interarrivals = np.diff(nhpp_arrivals)

    service_times = []

    env = simpy.Environment()
    servers = [[simpy.PriorityResource(env, capacity=1) for _ in range(n)] for _ in range(lanes_nb)]
    env.process(arrivals(env, servers, S, G,revenue, interarrivals, is_print))
    env.process(construction(env, servers, is_print))
    env.run()
    elapsed_time = np.array([G[x]-S[x] for x in G])
    speed = np.mean(1000/elapsed_time)
    time_of_arrival = np.array([S[x] for x in G])

    if is_print:
      print('Average time for a vehicle to pass the highway is: {:.2f} s'.format(np.mean(elapsed_time)))
      print('Average speed : {:.2f} m/s'.format(speed))
      print('Income : ${:.2f}'.format(sum(revenue)*360))
      print('Daily cost : ${:.2f}'.format(daily_cost))

    return sum(revenue)*360 - daily_cost, speed

np.random.seed(3)
revenue, avg_speed= simulate_with_speed(4)

In [ ]:
revenue = []
speed = []

# Comparisons between 2-6 lanes
for i in range(2,7):
    np.random.seed(42)
    revenue_i = []
    speed_i = []
    for j in range(10):
        r, s = simulate_with_speed(i)
        revenue_i.append(r)
        speed_i.append(s)
    revenue.append(np.mean(revenue_i))
    speed.append(np.mean(speed_i))

In [ ]:
revenue

[3621.2166471333258,
 3908.7665985192957,
 4196.316549905264,
 4068.5165012912307,
 3908.5414526771965]

In [ ]:
speed_kmh = [x * 3.6 for x in speed]
speed_kmh

[64.28811054935383,
 68.50631483800915,
 69.42204362154565,
 70.3643692035286,
 70.47253658826064]

Previous Cases of original models but with thinning process implemented to try and make things uniform from a time perspective: https://colab.research.google.com/drive/1JPoP5adXJJ-YOE1FnPP1wvulxtiLBEMa?usp=sharing